In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import
import sys

from pymoo.indicators.igd import IGD
from pymoo.indicators.hv import HV

In [2]:
condbased_res1000_orig = pd.read_csv("locomotec-res-condfuzz-python", delim_whitespace=True)
condbased_res250_orig = pd.read_csv("locomotec-history-condfuzz-250", delim_whitespace=True)
condbased_res500_orig = pd.read_csv("locomotec-history-condfuzz-500", delim_whitespace=True)
condbased_res750_orig = pd.read_csv("locomotec-history-condfuzz-750", delim_whitespace=True)

In [3]:
selected_cols_for_indicators = ['uvMissSurface','uvExcessHuman','fuzzOpTimes']

In [4]:
def invert_signs_for_metrics(df):
    df_new = df.copy()
    df_new.uvMissSurface *= -1
    df_new.uvExcessHuman *= -1
    return df_new

In [5]:
# These have the appropriate signs now
condbased_res1000 = invert_signs_for_metrics(condbased_res1000_orig)
condbased_res250 = invert_signs_for_metrics(condbased_res250_orig)
condbased_res500 = invert_signs_for_metrics(condbased_res500_orig)
condbased_res750 = invert_signs_for_metrics(condbased_res750_orig)

In [6]:
def max_point_from_front(pf):
    return np.array([np.max(pf.uvMissSurface), np.max(pf.uvExcessHuman), np.max(pf.fuzzOpTimes)])

In [7]:
def max_point_from_all_fronts(pf_list):
    max_points_2d = np.column_stack(list(map(lambda pf: max_point_from_front(pf), pf_list)))
    max_of_all = np.amax(max_points_2d, axis=1)
    return max_of_all

In [8]:
def front_to_numpy_array(front):
    return front[selected_cols_for_indicators].to_numpy()

In [9]:
max_point_from_front(condbased_res500)

array([ -0.   ,  -0.   , 267.353])

In [10]:
all_fronts = [condbased_res250, condbased_res500, condbased_res750, condbased_res1000]
max_point_from_all_fronts(all_fronts)

array([ -0.   ,  -0.   , 267.353])

In [15]:
def indicators_for_intermediate_fronts(fronts):
    # Reference point is obtained as the max point from all fronts
    ref_point = max_point_from_all_fronts(fronts)
    ref_front = front_to_numpy_array(condbased_res1000)

    for front_pandas in fronts:
        # Get front as numpy array
        front = front_to_numpy_array(front_pandas)
        hv_ind = HV(ref_point=ref_point)
        igd_ind = IGD(ref_front)
        hv_val = hv_ind(front)
        igd_val = igd_ind(front)
        print("HV = " + str(hv_val) + "IGD = " + str(igd_val))

In [16]:
indicators_for_intermediate_fronts(all_fronts)

HV = 1813.3033955400003IGD = 29.143584373001993
HV = 2094.581752499001IGD = 12.250625939046172
HV = 5072.54860814IGD = 3.565302314849478
HV = 7155.899359200001IGD = 0.0
